In [10]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
from datetime import datetime
import os
import glob 

In [5]:
start_time = datetime.now()

In [40]:
dir = 'Y:\\delete\\Australian Government Department of Health'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

In [41]:
ser = Service("C:/Users/AIMPAI0/Downloads/chromedriver.exe")
op = webdriver.ChromeOptions()
prefs = {"download.default_directory" : "Y:\\delete\\Australian Government Department of Health"}
op.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(service=ser, options=op)
driver.maximize_window()  
driver.get("https://www.industrialchemicals.gov.au/help-and-guides/list-chemicals-high-hazards-categorisation")
driver.implicitly_wait(5)
driver.find_element(by=By.XPATH, value='//*[@id="block-nicnas-theme-mainpagecontent"]/div/article/div/div[2]/div/div[1]/div[2]/div/div/div/article/div/a').click()
time.sleep(10)
driver.close()

In [46]:
path = 'Y:\\delete\\Australian Government Department of Health'
filenames = glob.glob(path + "/*.xlsx")
for f in filenames:
    dataset = pd.read_excel(f,sheet_name='Consolidated list')
#dataset = dataset[['CAS RN']]
dataset.rename(columns = {'CAS RN':'CAS Number'}, inplace = True)
dataset = dataset.loc[dataset['CAS Number'] != 'not specified']
dataset = dataset.applymap(lambda x: x.strip() if isinstance(x, str) else x)
CAS = pd.read_excel('Y:/delete/Chemical CAS.xlsx')
CAS['value'] = 'Not in list'
CAS = CAS.applymap(lambda x: x.strip() if isinstance(x, str) else x)

result = pd.merge(dataset, CAS, how="outer", on="CAS Number")

In [47]:
result['Jurisdiction'] = 'Australia'
result['RegulatoryAgency'] = 'Department of Health'
result['TypeOfRegulation'] = 'Health'
result['Criteria'] = 'hazard band C for human health or hazard bands D or C for environment'

result['Description'] = 'High Hazard Chemical'
result.loc[result['value'] == 'Not in list', 'Description'] = 'Not High Hazard Chemical'

result['HazardClassification'] = 'High'
result.loc[result['value'] == 'Not in list', 'HazardClassification'] = 'Low'

result['CauseOfClassification'] = 'On list of "Chemicals with High Hazard for Classification"'
result.loc[result['value'] == 'Not in list', 'CauseOfClassification'] = 'Not on list of "Chemicals with High Hazard for Classification'

result['SourceName'] = 'Australian Government Department of Health'
result['SourceURL'] = 'https://www.industrialchemicals.gov.au/help-and-guides/list-chemicals-high-hazards-categorisation'

result.head(2)

,CAS Number,value,Jurisdiction,RegulatoryAgency,TypeOfRegulation,Criteria,Description,HazardClassification,CauseOfClassification,SourceName,SourceURL
0,50-00-0,NaN,Australia,Department of Health,Health,hazard band C for human health or hazard bands...,High Hazard Chemical,High,"On list of ""Chemicals with High Hazard for Cla...",Australian Government Department of Health,https://www.industrialchemicals.gov.au/help-an...
1,50-06-6,NaN,Australia,Department of Health,Health,hazard band C for human health or hazard bands...,High Hazard Chemical,High,"On list of ""Chemicals with High Hazard for Cla...",Australian Government Department of Health,https://www.industrialchemicals.gov.au/help-an...


In [ ]:
result.to_csv('Y:/delete/Reugulatory Assessment.csv',index=False,mode='a',header=False)

In [48]:
end_time = datetime.now()
print('The total time taken is', (end_time - start_time))

The total time taken is 0:20:31.498526
